# Fremtidsrettet data
### Ved hjelp av API


Gruppen har brukt data fra [developer.yr.no](https://api.met.no/weatherapi/locationforecast/2.0/documentation). Daten viser vær for gitte plasser i den kommende uken. Den inkluderer blant annet temperatur, vind fart og lufttrykk. Gruppen har valgt å bruke en API fra nettsiden som støtter JSON-filer.
Daten viser være for gitte plasser i den kommende uken. 


In [1]:
import pandas as pd
import requests

Under har gruppen lagt tilgjengelig noen koordinater som kan være enkle å bruke.
Her er link til [Google Maps](https://www.google.com/maps) dersom du ønsker å hente inn egne koordinater. 

In [2]:
#Breddegrad og lengdegrad er hentet fra Google Maps
locations = [
    ("Stryn", 61.904275, 6.715641),
    ("Paris", 48.854289, 2.342042),
    ("London", 51.501814, -0.140605),
    ("Cape Town", -33.922348, 18.424031),
    ("New York", 40.710676, -74.006219)
]

#Lager en oversiktstabell
data = []
for name, lat, lon in locations:
    data.append([name, lat, lon])

df = pd.DataFrame(data, columns=["Plass", "Breddegrad", "Lengdegrad"])
df.head()


,Plass,Breddegrad,Lengdegrad
0,Stryn,61.904275,6.715641
1,Paris,48.854289,2.342042
2,London,51.501814,-0.140605
3,Cape Town,-33.922348,18.424031
4,New York,40.710676,-74.006219


# Databehandling
De følgende kodene under brukes til å identifisere hull i datasett

In [26]:
#Koden lager en JSON fil ut i fra koordinatene som blir puttet inn i linken 
def make_weatherJSON(lat, lon):
    url = f"https://api.met.no/weatherapi/locationforecast/2.0/compact?lat={lat}&lon={lon}"
    headers = {'User-Agent': 'my-weather-app/1.0'}  #nødvendig for Yr API
    response = requests.get(url, headers=headers)
    
    
    if response.status_code != 200:
        raise Exception(f"API call failed with status code {response.status_code}")
    

    data = response.json()
    return data


In [31]:
#Rydder manglende verdier med fillna()
def clean_weather_data(data):
    timeseries = data["properties"]["timeseries"]

    lst = []
    for entry in timeseries:
        time = entry["time"]
        details = entry["data"]["instant"]["details"]
        lst.append({
            "Time": time,
            "Temperature": details.get("air_temperature"),
            "Rain": entry["data"].get("next_1_hours", {}).get("details", {}).get("precipitation_amount"),
            "Medium wind": details.get("wind_speed")
        })

    
    df = pd.DataFrame(lst)
        
    #gjør om "Tid" til datetime
    df["Time"] = pd.to_datetime(df["Time"], errors="coerce")

    #fyller inn evt. hull (NaN) med gjennomsnitt
    df.fillna(df.mean(numeric_only=True), inplace=True)
    
    return df

clean_weather_data(make_weatherJSON(61.904275, 6.715641))

,Time,Temperature,Rain,Medium wind
0,2025-03-19 14:00:00+00:00,7.3,0.000000,1.8
1,2025-03-19 15:00:00+00:00,7.2,0.000000,1.7
2,2025-03-19 16:00:00+00:00,6.6,0.000000,1.8
3,2025-03-19 17:00:00+00:00,5.3,0.000000,1.6
4,2025-03-19 18:00:00+00:00,3.3,0.000000,1.4
...,...,...,...,...
79,2025-03-28 12:00:00+00:00,9.5,0.007692,0.9
80,2025-03-28 18:00:00+00:00,6.0,0.007692,0.9
81,2025-03-29 00:00:00+00:00,3.8,0.007692,0.9
82,2025-03-29 06:00:00+00:00,3.1,0.007692,1.1


In [27]:
#sjekker om det er noen hull i datasettet etterpå
def check_NaN_counter(df):
    missing_counts = df.isna().sum()
    print("Antall NaN-verdier per kolonne:")
    print(missing_counts)
    return missing_counts

data = make_weatherJSON(61.904275, 6.715641)
df = clean_weather_data(data)
check_NaN_counter(df)

Antall NaN-verdier per kolonne:
Time              0
Temperature       0
Rain              0
Wind direction    0
Medium wind       0
dtype: int64


Time              0
Temperature       0
Rain              0
Wind direction    0
Medium wind       0
dtype: int64

Videre under blir hullene i datasettet behandlet

In [ ]:
#MANGLER LIST COMPERHENTSION 
#MANGLER PANDAS SQL